In [5]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
import tensorflow as tf

In [6]:
(train_x, train_y), (test_x, test_y) = tf.keras.datasets.mnist.load_data()

In [7]:
train_x = train_x / 255. 
test_x = test_x / 255.

In [8]:
train_y

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

In [41]:
train_ds = tf.data.Dataset.from_tensor_slices((train_x, train_y))
train_ds = train_ds.shuffle(1000).batch(32)
# train_ds = t

test_ds = tf.data.Dataset.from_tensor_slices((test_x, test_y))
test_ds = test_ds.batch(32)

In [42]:
inputs = tf.keras.Input((28, 28, 1))

net = tf.keras.layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
net = tf.keras.layers.Activation('relu')(net)
net = tf.keras.layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
net = tf.keras.layers.Activation('relu')(net)
net = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(net)
net = tf.keras.layers.Dropout(0.25)(net) 
# net = 

net = tf.keras.layers.Conv2D(64, (3, 3), padding='SAME')(inputs)
net = tf.keras.layers.Activation('relu')(net)
net = tf.keras.layers.Conv2D(64, (3, 3), padding='SAME')(inputs)
net = tf.keras.layers.Activation('relu')(net)
net = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(net)
net = tf.keras.layers.Dropout(0.25)(net) 

net = tf.keras.layers.Flatten()(net) 
net = tf.keras.layers.Dense(512)(net)
net = tf.keras.layers.Activation('relu')(net) 
net = tf.keras.layers.Dropout(0.5)(net)
net = tf.keras.layers.Dense(10)(net)
net = tf.keras.layers.Activation('softmax')(net)


model = tf.keras.Model(inputs=inputs, outputs=net)

# net = tf.keras.layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
# net = tf.keras.layers.Activation('relu')(net)
# net = tf.keras.layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
# net = tf.keras.layers.Activation('relu')(net)
# net = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(net)
# net = tf.keras.layers.Dropout(0.25)(net) 


In [43]:
model.compile(optimizer=tf.keras.optimizers.Adam(0.01)
            , loss=tf.keras.losses.SparseCategoricalCrossentropy()
            , metrics=['accuracy']
)

In [44]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy() 
optimizer=tf.keras.optimizers.Adam(0.01)
# hist = model.fit(train_ds, epochs=10)

In [45]:
train_loss = tf.keras.metrics.Mean() 
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy() 

test_loss = tf.keras.metrics.Mean() 
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy() 


In [46]:
@tf.function 
def train_step(image, label): 
    with tf.GradientTape() as tape: 
        prediction = model(image) 
        loss = loss_object(label, prediction) 
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(label, prediction) 

In [47]:
@tf.function 
def test_step(image, label): 
    # with tf.GradientTape() as tape: 
    prediction = model(image) 
    loss = loss_object(label, prediction) 
    # gradients = tape.gradient(loss, model.trainable_variables)
    # optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    test_loss(loss)
    test_accuracy(label, prediction) 

In [49]:
for epoch in range(100):
    
    for image, label in train_ds: 
        train_step(image, label) 
        
    for image, label in test_ds: 
        test_step(image, label) 


    temp = "Epoch : {}, loss : {}, accuracy : {}, test_loss : {}, test_accuracy : {}"

    print(temp.format(epoch + 1
        , train_loss.result() 
        , train_accuracy.result() * 100 
        , test_loss.result() 
        , test_accuracy.result() * 100)
    )

Epoch : 1, loss : 0.07042494416236877, accuracy : 98.07889556884766, test_loss : 0.1201811134815216, test_accuracy : 97.32166290283203
Epoch : 2, loss : 0.06623753905296326, accuracy : 98.21503448486328, test_loss : 0.13078251481056213, test_accuracy : 97.34713745117188
Epoch : 3, loss : 0.06342901289463043, accuracy : 98.32191467285156, test_loss : 0.13587358593940735, test_accuracy : 97.35250091552734
Epoch : 4, loss : 0.06056232750415802, accuracy : 98.42269134521484, test_loss : 0.14349465072155, test_accuracy : 97.39778137207031
Epoch : 5, loss : 0.0586961954832077, accuracy : 98.50527954101562, test_loss : 0.15035317838191986, test_accuracy : 97.39299774169922
Epoch : 6, loss : 0.05657706782221794, accuracy : 98.5822525024414, test_loss : 0.16388638317584991, test_accuracy : 97.34545135498047
Epoch : 7, loss : 0.05528433993458748, accuracy : 98.64360809326172, test_loss : 0.1882493495941162, test_accuracy : 97.34166717529297
Epoch : 8, loss : 0.054372936487197876, accuracy : 98.7